# Scaling up ML using Cloud ML Engine 

We discuss and develop a TensorFlow model to predict churn-rate of a fictional bank and package it up so that it can be run in Cloud MLE. For now, we'll run this on a small dataset. The model that was developed is rather simplistic, and therefore, the accuracy of the model is not great either.  However, this notebook illustrates how to package up a TensorFlow model to run it within Cloud ML.

In [1]:
import tensorflow as tf
print tf.__version__

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.5.0


### Environment variables for project and bucket </h2>

Note that:

1. Your project id is the *unique* string that identifies your project (not the project name). You can find this from the GCP Console dashboard's Home page.  My dashboard reads:  <b>Project ID:</b> tough-transport-195311 </li>


2. Cloud training often involves saving and restoring model files. Therefore, we should <b>create a single-region bucket</b>. If you don't have a bucket already, I suggest that you create one from the GCP console (because it will dynamically check whether the bucket name you want is available).


In [2]:
import os
PROJECT = 'tough-transport-195311' # REPLACE WITH YOUR PROJECT ID
BUCKET = 'loonycorn_mle' # REPLACE WITH YOUR BUCKET NAME
REGION = 'us-central1' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1

In [3]:
import os
REPO = "/content/datalab/tfdeployment"
os.listdir(REPO)

['preprocessing.ipynb',
 'churn_trained',
 '.ipynb_checkpoints',
 'preprocessing-Copy1.ipynb',
 'cloudmle',
 'datasets']

In [4]:
# for bash
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['REPO'] = REPO

In [5]:
%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


Allow the Cloud ML Engine service account to read/write to the bucket containing training data. Note that the cell below requires Google Cloud Machine Learning Engine to be enabled in the project of choice.

In [6]:
%bash
PROJECT_ID=$PROJECT
AUTH_TOKEN=$(gcloud auth print-access-token)
SVC_ACCOUNT=$(curl -X GET -H "Content-Type: application/json" \
    -H "Authorization: Bearer $AUTH_TOKEN" \
    https://ml.googleapis.com/v1/projects/${PROJECT_ID}:getConfig \
    | python -c "import json; import sys; response = json.load(sys.stdin); \
    print response['serviceAccount']")

echo "Authorizing the Cloud ML Service account $SVC_ACCOUNT to access files in $BUCKET"
gsutil -m defacl ch -u $SVC_ACCOUNT:R gs://$BUCKET
gsutil -m acl ch -u $SVC_ACCOUNT:R -r gs://$BUCKET  # error message (if bucket is empty) can be ignored
gsutil -m acl ch -u $SVC_ACCOUNT:W gs://$BUCKET 

Authorizing the Cloud ML Service account service-60643179177@cloud-ml.google.com.iam.gserviceaccount.com to access files in loonycorn_mle


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   230    0   230    0     0    813      0 --:--:-- --:--:-- --:--:--   815
No changes to gs://loonycorn_mle/
No changes to gs://loonycorn_mle/churn_prediction/inputs/Churn_Modelling.csv
No changes to gs://loonycorn_mle/churn_prediction/inputs/churn-test.csv
No changes to gs://loonycorn_mle/churn_prediction/inputs/churn-train.csv
No changes to gs://loonycorn_mle/churn_prediction/inputs/churn-valid.csv
Updated ACL on gs://loonycorn_mle/churn_prediction/inputs/churn_trained/
Updated ACL on gs://loonycorn_mle/churn_prediction/inputs/churn_trained/eval/
Updated ACL on gs://loonycorn_mle/churn_prediction/inputs/churn_trained/checkpoint
Updated ACL on gs://loonycorn_mle/churn_prediction/inputs/churn_trained/eval/events.out.tfevents.1521540069.cmle-training-2546897926222511384
Updated ACL on gs://loonycorn_mle/churn_prediction/inputs/

### Packing up the code

Take your code and put into a standard Python package structure.<a href="churn_prediction/trainer/model.py">model.py</a> and <a href="churn_prediction/trainer/task.py">task.py</a> contain the Tensorflow code from earlier (explore the <a href="churn_prediction/trainer/">directory structure</a>).

In [17]:
! find churn_prediction

churn_prediction
churn_prediction/setup.py
churn_prediction/trainer
churn_prediction/trainer/model.py
churn_prediction/trainer/task.py
churn_prediction/trainer/__init__.py
churn_prediction/.ipynb_checkpoints


### Find absolute path to your data

Note the absolute paths below. /content is mapped in Datalab to where the home icon takes you

In [18]:
%bash
rm -rf $REPO/churn_trained
head -1 $REPO/datasets/churn-train.csv
head -1 $REPO/datasets/churn-valid.csv

0,734,1,0,47,10,91522.04,2,1,1,138835.91,0
0,646,0,0,34,3,131283.11,1,0,0,130500.65,0
